**Indiquer dans la boîte ci-dessous vos noms et prénoms**

Dousble-cliquer ici et remplacer par les noms et prénoms de votre binôme

# 3I026 - Intelligence artificielle & Data science

# Projet

## Données

Vous devez travailler sur les données du projet Kaggle *European Soccer Database* accessible à l'adresse https://www.kaggle.com/hugomathien/soccer.

Les données en version SQLite sur la page http://www-connex.lip6.fr/~schwander/3I026/soccer.zip ou en version CSV http://www-connex.lip6.fr/~denoyer/wordpress/wp-content/uploads/2018/01/soccer.tar.gz. Pour facilter le travail dans les salles de la PPTI (et éviter de dépasser le quota de votre home), les données sont accessibles aussi dans `/Vrac/3I026/soccer/`.

Quel que soit le moyen pour récupérer le fichier de données, il est indispensable de lire en détail la page Kaggle pour comprendre à quoi ces données correspondent.

**Rappel:** il est attendu un notebook Python par problème, et le problème étudié doit être décrit précisément. Votre rapport devra impérativement suivre le format suivant.

## Partie 1 - Description du problème

Le but de ce projet est de déterminer le résultat d'un match de champions league.
Nous avons à notre disposition les données des match de ligue des 11 pays qui participe à la champion's league depuis 2008, les données des équipes et leurs caractéristiques et les données des joueurs et leurs caractéristiques.
Pour procéder à notre programme de prédiction nous avons pensés à utiliser un labelset incluant le score, les joueurs et leur potentiel, le potentiel des équipes et les  


## Partie 2 - Modèle

In [1]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect("database.sqlite")
conn.row_factory = sqlite3.Row

c = conn.cursor()

teams = pd.read_sql_query("Select * from Team", conn)
team_attributes = pd.read_sql_query("Select * from Team_Attributes", conn)
countries = pd.read_sql_query("Select * from Country", conn)
league = pd.read_sql_query("Select * from League", conn)
match = pd.read_sql_query("Select * from Match", conn)
players = pd.read_sql_query("Select * from Player", conn)
player_attributes = pd.read_sql_query("Select * from Player_attributes", conn)

conn.close()


In [2]:
teams.head(5)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [3]:
team_attributes.fillna(team_attributes.mean(), inplace=True)

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,48.607362,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.000000,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.000000,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,48.607362,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,48.607362,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
5,6,77,8485,2012-02-22 00:00:00,58,Balanced,48.607362,Little,62,Mixed,...,55,Normal,Organised,40,Medium,40,Press,60,Normal,Cover
6,7,77,8485,2013-09-20 00:00:00,62,Balanced,48.607362,Little,45,Mixed,...,55,Normal,Organised,42,Medium,42,Press,60,Normal,Cover
7,8,77,8485,2014-09-19 00:00:00,58,Balanced,64.000000,Normal,62,Mixed,...,57,Normal,Organised,41,Medium,42,Press,60,Normal,Cover
8,9,77,8485,2015-09-10 00:00:00,59,Balanced,64.000000,Normal,53,Mixed,...,63,Normal,Free Form,49,Medium,45,Press,63,Normal,Cover
9,10,614,8576,2010-02-22 00:00:00,60,Balanced,48.607362,Little,40,Mixed,...,55,Normal,Organised,30,Deep,70,Double,30,Narrow,Offside Trap


In [4]:
team_attributes.describe()

,id,team_fifa_api_id,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
count,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000
mean,729.500000,17706.982167,9995.727023,52.462277,48.607362,48.490398,52.165295,53.731824,53.969136,46.017147,49.251029,52.185871
std,421.032659,39179.857739,13264.869900,11.545869,5.601170,10.896101,10.360793,11.086796,10.327566,10.227225,9.738028,9.574712
min,1.000000,1.000000,1601.000000,20.000000,24.000000,20.000000,21.000000,20.000000,22.000000,23.000000,24.000000,29.000000
25%,365.250000,110.000000,8457.750000,45.000000,48.607362,40.000000,46.000000,47.000000,48.000000,39.000000,44.000000,47.000000
50%,729.500000,485.000000,8674.000000,52.000000,48.607362,50.000000,52.000000,53.000000,53.000000,45.000000,48.000000,52.000000
75%,1093.750000,1900.000000,9904.000000,62.000000,48.607362,55.000000,59.000000,62.000000,61.000000,51.000000,55.000000,58.000000
max,1458.000000,112513.000000,274581.000000,80.000000,77.000000,80.000000,80.000000,80.000000,80.000000,72.000000,72.000000,73.000000


In [5]:
countries

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [6]:
league

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [7]:
match.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [8]:
match.describe()[["home_player_X11"]]

,home_player_X11
count,24147.000000
mean,5.783244
std,0.759585
min,1.000000
25%,5.000000
50%,6.000000
75%,6.000000
max,7.000000


In [9]:
players.head(5)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [10]:
player_attributes.head(5)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [11]:
player_attributes.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


In [12]:
match_stat=match.copy()
match_stat=match_stat[["league_id","match_api_id", "home_team_api_id", "away_team_api_id", "home_team_goal", "away_team_goal"]]
match_stat.head(5)

,league_id,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,492473,9987,9993,1,1
1,1,492474,10000,9994,0,0
2,1,492475,9984,8635,0,3
3,1,492476,9991,9998,5,0
4,1,492477,7947,9985,1,3


In [13]:
team_match_stat=teams.copy()
home_stat = match_stat.groupby("home_team_api_id").mean()[["league_id", "home_team_goal"]].reset_index()
away_stat = match_stat.groupby("away_team_api_id").mean()[[ "away_team_goal"]].reset_index()

team_match_stat = team_match_stat.merge(home_stat, left_on="team_api_id", right_on="home_team_api_id")
team_match_stat = team_match_stat.merge(away_stat, left_on="team_api_id", right_on="away_team_api_id")
team_match_stat = team_match_stat.drop("home_team_api_id",1)
team_match_stat = team_match_stat.drop("away_team_api_id",1)
team_match_stat.head(5)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name,league_id,home_team_goal,away_team_goal
0,1,9987,673.0,KRC Genk,GEN,1.0,1.933962,1.349057
1,2,9993,675.0,Beerschot AC,BAC,1.0,1.473684,0.815789
2,3,10000,15005.0,SV Zulte-Waregem,ZUL,1.0,1.660377,1.226415
3,4,9994,2007.0,Sporting Lokeren,LOK,1.0,1.405660,1.226415
4,5,9984,1750.0,KSV Cercle Brugge,CEB,1.0,1.307692,1.032967


In [14]:
home = team_match_stat["home_team_goal"].values
away = team_match_stat["away_team_goal"].values
goal_rate = (home+away)/2

team_match_stat["goal_rate"]=pd.Series(goal_rate)
team_match_stat.head(5)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name,league_id,home_team_goal,away_team_goal,goal_rate
0,1,9987,673.0,KRC Genk,GEN,1.0,1.933962,1.349057,1.641509
1,2,9993,675.0,Beerschot AC,BAC,1.0,1.473684,0.815789,1.144737
2,3,10000,15005.0,SV Zulte-Waregem,ZUL,1.0,1.660377,1.226415,1.443396
3,4,9994,2007.0,Sporting Lokeren,LOK,1.0,1.405660,1.226415,1.316038
4,5,9984,1750.0,KSV Cercle Brugge,CEB,1.0,1.307692,1.032967,1.170330


In [15]:
team_match_stat = team_match_stat.sort_values(["goal_rate"], ascending=[0])
team_match_stat.loc[team_match_stat["league_id"]==24558]

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name,league_id,home_team_goal,away_team_goal,goal_rate
287,49118,9931,896.0,FC Basel,BAS,24558.0,2.405594,1.923077,2.164336
286,49117,10192,900.0,BSC Young Boys,YB,24558.0,2.230769,1.468531,1.849650
293,49124,10243,894.0,FC Zürich,ZUR,24558.0,1.685315,1.587413,1.636364
284,49115,9956,322.0,Grasshopper Club Zürich,GRA,24558.0,1.636364,1.363636,1.500000
290,49121,10199,897.0,FC Luzern,LUZ,24558.0,1.671329,1.272727,1.472028
292,49123,7955,435.0,Neuchâtel Xamax,XAM,24558.0,1.682540,1.031746,1.357143
295,49837,10191,1715.0,FC Thun,THU,24558.0,1.635514,0.971963,1.303738
289,49120,10179,110770.0,FC Sion,SIO,24558.0,1.538462,1.055944,1.297203
294,49479,10190,898.0,FC St. Gallen,GAL,24558.0,1.416667,1.138889,1.277778
298,51606,7896,NaN,Lugano,LUG,24558.0,1.388889,1.166667,1.277778


In [16]:
team_attributes.loc[team_attributes["team_api_id"]==9931].describe()

,id,team_fifa_api_id,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
count,6.000000,6.0,6.0,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,133.500000,896.0,9931.0,55.000000,53.404908,44.833333,60.833333,63.833333,52.500000,47.666667,57.500000,54.500000
std,1.870829,0.0,0.0,9.486833,7.432326,2.401388,8.010410,4.833908,2.345208,7.033254,2.345208,2.810694
min,131.000000,896.0,9931.0,40.000000,48.607362,40.000000,50.000000,54.000000,49.000000,38.000000,53.000000,53.000000
25%,132.250000,896.0,9931.0,49.750000,48.607362,45.250000,54.750000,65.250000,51.000000,47.000000,58.000000,53.000000
50%,133.500000,896.0,9931.0,61.000000,48.607362,46.000000,66.000000,66.000000,54.000000,47.000000,58.000000,53.000000
75%,134.750000,896.0,9931.0,61.000000,59.401840,46.000000,66.000000,66.000000,54.000000,47.000000,58.000000,54.500000
max,136.000000,896.0,9931.0,61.000000,63.000000,46.000000,66.000000,66.000000,54.000000,60.000000,60.000000,60.000000


In [17]:
match_stat=match.copy()
match_stat=match_stat.loc[match_stat["date"]>="2015"]


def add_row(matchSet, labeledset):
    
    if(matchSet["home_team_goal"]==matchSet["away_team_goal"]):
        return False
    liste_home=[]
    
    ateam=team_attributes.loc[team_attributes["team_api_id"]==matchSet["away_team_api_id"]].loc[team_attributes["date"]<=matchSet["date"]].mean()
    hteam=team_attributes.loc[team_attributes["team_api_id"]==matchSet["home_team_api_id"]].loc[team_attributes["date"]<=matchSet["date"]].mean()
    
    if(ateam.isnull().any().any() or hteam.isnull().any().any()):
        return False
    """
    liste_home.append(matchSet["home_player_X1"])
    liste_home.append(matchSet["home_player_Y1"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_1"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X2"])
    liste_home.append(matchSet["home_player_Y2"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_2"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X3"])
    liste_home.append(matchSet["home_player_Y3"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_3"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X4"])
    liste_home.append(matchSet["home_player_Y4"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_4"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X5"])
    liste_home.append(matchSet["home_player_Y5"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_5"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X6"])
    liste_home.append(matchSet["home_player_Y6"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_6"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X7"])
    liste_home.append(matchSet["home_player_Y7"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_7"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X8"])
    liste_home.append(matchSet["home_player_Y8"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_8"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X9"])
    liste_home.append(matchSet["home_player_Y9"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_9"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X10"])
    liste_home.append(matchSet["home_player_Y10"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_10"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    
    liste_home.append(matchSet["home_player_X11"])
    liste_home.append(matchSet["home_player_Y11"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["home_player_11"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_home.append(player.loc["overall_rating"])
    liste_home.append(player.loc["potential"])
    """
    #liste_home.append(team_match_stat.loc[team_match_stat["team_api_id"]==matchSet["home_team_api_id"]].mean().loc["home_team_goal"])
    liste_home.append(hteam.loc["buildUpPlaySpeed"])
    liste_home.append(hteam.loc["buildUpPlayDribbling"])
    liste_home.append(hteam.loc["buildUpPlayPassing"])
    liste_home.append(hteam.loc["chanceCreationPassing"])
    liste_home.append(hteam.loc["chanceCreationCrossing"])
    liste_home.append(hteam.loc["chanceCreationShooting"])
    liste_home.append(hteam.loc["defencePressure"])
    liste_home.append(hteam.loc["defenceAggression"])
    liste_home.append(hteam.loc["defenceTeamWidth"])
    
    

    liste_away=[]
    """
    liste_away.append(matchSet["away_player_X1"])
    liste_away.append(matchSet["away_player_Y1"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_1"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X2"])
    liste_away.append(matchSet["away_player_Y2"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_2"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X3"])
    liste_away.append(matchSet["away_player_Y3"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_3"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X4"])
    liste_away.append(matchSet["away_player_Y4"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_4"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X5"])
    liste_away.append(matchSet["away_player_Y5"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_5"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X6"])
    liste_away.append(matchSet["away_player_Y6"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_6"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X7"])
    liste_away.append(matchSet["away_player_Y7"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_7"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X8"])
    liste_away.append(matchSet["away_player_Y8"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_8"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X9"])
    liste_away.append(matchSet["away_player_Y9"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_9"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X10"])
    liste_away.append(matchSet["away_player_Y10"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_10"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    
    liste_away.append(matchSet["away_player_X11"])
    liste_away.append(matchSet["away_player_Y11"])
    player=player_attributes.loc[player_attributes["player_api_id"]==matchSet["away_player_11"]].loc[player_attributes["date"]<=matchSet["date"]].mean()
    liste_away.append(player.loc["overall_rating"])
    liste_away.append(player.loc["potential"])
    """
    #liste_away.append(team_match_stat.loc[team_match_stat["team_api_id"]==matchSet["away_team_api_id"]].mean().loc["away_team_goal"])
    liste_away.append(ateam.loc["buildUpPlaySpeed"])
    liste_away.append(ateam.loc["buildUpPlayDribbling"])
    liste_away.append(ateam.loc["buildUpPlayPassing"])
    liste_away.append(ateam.loc["chanceCreationPassing"])
    liste_away.append(ateam.loc["chanceCreationCrossing"])
    liste_away.append(ateam.loc["chanceCreationShooting"])
    liste_away.append(ateam.loc["defencePressure"])
    liste_away.append(ateam.loc["defenceAggression"])
    liste_away.append(ateam.loc["defenceTeamWidth"])
    
    if(matchSet["home_team_goal"]>matchSet["away_team_goal"]):
        labeledset.addExample(liste_home,1)
        labeledset.addExample(liste_away,-1)
    else:
        labeledset.addExample(liste_home,-1)
        labeledset.addExample(liste_away,1)
        
    return True

In [18]:
def add_row2(matchSet, labeledset):
    
    if(matchSet["home_team_goal"]==matchSet["away_team_goal"]):
        return False
    liste=[]
    
    ateam=team_attributes.loc[team_attributes["team_api_id"]==matchSet["away_team_api_id"]].loc[team_attributes["date"]<=matchSet["date"]].mean()
    hteam=team_attributes.loc[team_attributes["team_api_id"]==matchSet["home_team_api_id"]].loc[team_attributes["date"]<=matchSet["date"]].mean()
    
    if(ateam.isnull().any().any() or hteam.isnull().any().any()):
        return False

    
    liste.append(hteam.loc["buildUpPlaySpeed"])
    liste.append(hteam.loc["buildUpPlayDribbling"])
    liste.append(hteam.loc["buildUpPlayPassing"])
    liste.append(hteam.loc["chanceCreationPassing"])
    liste.append(hteam.loc["chanceCreationCrossing"])
    liste.append(hteam.loc["chanceCreationShooting"])
    liste.append(hteam.loc["defencePressure"])
    liste.append(hteam.loc["defenceAggression"])
    liste.append(hteam.loc["defenceTeamWidth"])
    

    
    liste.append(ateam.loc["buildUpPlaySpeed"])
    liste.append(ateam.loc["buildUpPlayDribbling"])
    liste.append(ateam.loc["buildUpPlayPassing"])
    liste.append(ateam.loc["chanceCreationPassing"])
    liste.append(ateam.loc["chanceCreationCrossing"])
    liste.append(ateam.loc["chanceCreationShooting"])
    liste.append(ateam.loc["defencePressure"])
    liste.append(ateam.loc["defenceAggression"])
    liste.append(ateam.loc["defenceTeamWidth"])
    
    if(matchSet["home_team_goal"]>matchSet["away_team_goal"]):
        labeledset.addExample(liste,1)
    else:
        labeledset.addExample(liste,-1)
        
    return True

les données à entrer dans un labeledset 

In [19]:
def matchToLabeledset(nb_data ,match):
    
    k=0
    train=LabeledSet(18)
    test=LabeledSet(18)
    
    for i in match.index:
        if(k>nb_data*2):
            break
        if(k%2==0):
            if(add_row2(match.loc[i], train)):
                k+=1
        else:
            if(add_row2(match.loc[i], test)):
                k+=1
    
    return train, test

In [20]:
matchSet=match_stat.loc[1543]
team_match_stat.loc[team_match_stat["team_api_id"]==matchSet["away_team_api_id"]].mean().loc["away_team_goal"]

1.0219780219780219

## Partie 3 - Code

In [21]:
from perceptron import LabeledSet
from perceptron import Classifier

matchset_train, matchset_test = matchToLabeledset(500, match_stat)

In [24]:
from perceptron import Perceptron

percepmatch=Perceptron(18,0.03,1000)
percepmatch.train(matchset_train)  
percepmatch.accuracy(matchset_test)

C:\Users\togos\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


0.61

In [25]:
from perceptron import KNN

knnmatch=KNN(18,5)
knnmatch.train(matchset_train)  
knnmatch.accuracy(matchset_test)

0.62

In [26]:
from perceptron import ClassifierBaggingPerceptron

bagPerc = ClassifierBaggingPerceptron(18, 30, 0.2, True)
bagPerc.train(matchset_train)
print("Accuracy Out of Bags: ",1.0-bagPerc.outOfBagsError())
print("Accuracy: ",bagPerc.accuracy(matchset_test))

C:\Users\togos\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


Accuracy Out of Bags:  0.9920159680638723
Accuracy:  0.578


In [30]:
from perceptron import ClassifierBaggingKNN

bagKnn = ClassifierBaggingKNN(18, 30, 0.2, True)
bagKnn.train(matchset_train)
print("Accuracy Out of Bags: ",1.0-bagKnn.outOfBagsError())
print("Accuracy: ",bagKnn.accuracy(matchset_test))

Accuracy Out of Bags:  0.9900199600798403
Accuracy:  0.63


## Partie 4 - Protocole expérimental

## Partie 5 - Résultats

## Partie 6 - Analyse